# Neural Network

Neural Network 近十年最有突破的技术，虽然不是万能的，但是在解决很多问题上，已经表现出了划时代的意义。Neural Network 由很多层 layer 组成，每一层 layer 又由很多个 cell 组成。一个 cell 可以是一个简单的 Logistic Regression Model 也可以是 Linear Regression Model，还可以是其他复杂的模型。最近很多年大家在做的事情主要就是改变 layer 的连接结构，改变 cell 所采用的模型，改变 Neural Network 所采用的损失函数。

我个人觉得，近期 Neural Network 发展过程中最重要的一个技术，就是自动求导，大大的降低了 Neural Network 的实现成本，使得人们可以快速的实现自己的想法，验证最后的结果。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


class Dnn(nn.Module):
    def __init__(self, layers_parameter, epoch=100, learning_rate=1e-3, optimizer='SGD', verbose=False, batch_size=32):
        super(Dnn, self).__init__()
        self.layers_parameter = layers_parameter
        self.epoch = epoch
        self.optimizer = optimizer
        self.loss_fn = nn.BCELoss(reduction='mean')
        self.verbose = verbose
        self.batch_size = batch_size
        self.dnn = None
        self.learning_rate = learning_rate

    def fit(self, X, y):
        if self.dnn is None:
            self.dnn = nn.Sequential(nn.Linear(X.shape[1], self.layers_parameter[0]).double(),
                                     nn.ReLU(),
                                     nn.BatchNorm1d(self.layers_parameter[0]).double(),
                                     nn.Linear(self.layers_parameter[0], self.layers_parameter[1]).double(),
                                     nn.ReLU(),
                                     nn.Linear(self.layers_parameter[1], self.layers_parameter[2]).double(),
                                     nn.ReLU(),
                                     nn.Linear(self.layers_parameter[2], self.layers_parameter[3]).double(),
                                     nn.ReLU(),
                                     nn.Linear(self.layers_parameter[3], 1).double(),
                                     nn.Sigmoid())

            if self.optimizer == 'Adam':
                self.optimizer = optim.Adam(self.parameters(), lr=self.learning_rate, amsgrad=True)
            elif self.optimizer == 'SGD':
                self.optimizer = optim.SGD(self.parameters(), lr=self.learning_rate, weight_decay=1e-5, momentum=0.1,
                                           nesterov=True)

        for epoch in range(self.epoch):
            start = 0
            end = start + self.batch_size
            while start < X.shape[0]:

                if end >= X.shape[0]:
                    end = X.shape[0]

                X_batch = torch.from_numpy(X[start:end, :])
                y_batch = torch.from_numpy(y[start:end]).reshape(1, end - start)
                y_batch_pred = self.forward(X_batch).reshape(1, end - start)

                loss = self.loss_fn(y_batch_pred, y_batch)
                loss.backward()
                self.optimizer.step()
                start = end
                end += self.batch_size

            if self.verbose and epoch % (self.epoch / 20) == 0:
                print('EPOCH: %d, loss: %f' % (epoch, loss))
        return self

    def forward(self, X):
        if self.dnn is not None:
            return self.dnn(X)
        else:
            print('You should fit first!')
            return self

    def predict_proba(self, X):
        X = torch.from_numpy(X)
        if self.dnn is not None:
            return self.dnn(X)
        else:
            print('You should fit first!')
            return self

    def predict(self, X):
        X = torch.from_numpy(X)
        if self.dnn is not None:
            out = self.dnn(X)
            out[out >= 0.5] = 1.0
            out[out < 0.5] = 0.0
            return out
        else:
            print('You should fit first!')
            return self
